## Segmenting and clustering neighbourhoods in Toronto
![alt text](https://png2.kisspng.com/sh/881afc8b1b85feadbe692e5b19c09418/L0KzQYm3UsIzN6ZpiZH0aYP2gLBuTgRwepDzjNG2YYLmeLr7hfN1faNqRdtsb36wdLb6ifdvNZpxhOd8dILkhLr2jr11d6N0huZ4LXPshMq0gwVqdJVuhtk2ZoLodX77j71xfZ1xReZxZT3wccXskvlidF46eapuZUK7doGAVcYxOF85S6gEMkC2RIK8Uck0OWQ6TaM5M0C0PsH1h5==/kisspng-toronto-architecture-icon-design-illustration-toronto-city-building-free-to-pull-the-material-5a8ee28f075600.4369203415193135510301.png)
___
### Notebook contents:
#### 1. Web scraping and creating a *pandas* dataframe
#### 2. Adding geographical coordinates and merging into a new dataframe
#### 3. Exploring and clustering neighbourhoods of Toronto using *Foursquare*

---
### 1. Web scraping and creating a *pandas* dataframe
**First of all, download all the required libraries:**

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from bs4 import BeautifulSoup as bs

print('Libraries imported.')

Libraries imported.


**Use *BeautifulSoup* to scrape the following web page**

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = bs(source, 'lxml')
table_data = soup.find('table')

**Create a *pandas* dataframe from website**

In [3]:
PostalCode = []
Borough = []
Neighborhood = []

for row in table_data.find_all('tr'):
    count = 0
    columns = row.find_all('td')
    for column in columns:
            count += 1
            if (count == 1):
               PostalCode.append(column.get_text())
            elif (count == 2):
                Borough.append(column.get_text())
            else:
                Neighborhood.append(column.get_text())
Neighborhood = list(map(lambda x:x.strip(),Neighborhood))
dictionary = {'PostalCode':PostalCode,'Borough':Borough,'Neighborhood':Neighborhood}
table = pd.DataFrame(dictionary) 
print(table.shape)
table.head()

(289, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


**Ignore cells with a borough that is *'Not assigned'***

In [4]:
dropborough_table = table[table.Borough != 'Not assigned']
dropborough_table.reset_index(drop=True,inplace = True)
dropborough_table.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [5]:
dropborough_table.shape

(212, 3)

**Join neighbourhoods in the same postal code area and separate by commas**

In [6]:
dropborough_table = dropborough_table.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

**Replace neighbourhoods that are *'Not assigned'* with the name of the borough**

In [7]:
mask = dropborough_table.Neighborhood =='Not assigned'
dropborough_table.loc[mask,'Neighborhood'] = dropborough_table.loc[mask,'Borough']
dropborough_table.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [8]:
dropborough_table.shape

(103, 3)

### 2. Adding geographical coordinates and merging into a new dataframe
**Import csv file to get the latitude and longitude coordinates of each neighbourhood**

In [9]:
geo_coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
geo_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Merging tables into a new *pandas* dataframe**

In [10]:
merged_table = [dropborough_table,geo_coordinates]
Toronto_neigh = pd.concat(merged_table,axis=1)
Toronto_neigh.drop(columns =['Postal Code'],inplace = True)
print(Toronto_neigh.shape)
Toronto_neigh.head(10)

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### 3. Exploring and clustering neighbourhoods of Toronto using Foursquare

**Define a user_agent and find geo coordinates for Toronto**

In [11]:
address = 'Toronto'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


**Create and display a folium map of Toronto**

In [12]:
# create a map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_neigh['Latitude'], Toronto_neigh['Longitude'], Toronto_neigh['Borough'], Toronto_neigh['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [13]:
#save map as html
map_toronto.save(outfile= "toronto_map.html")

**Create a new dataframe (only neighbourhoods in boroughs that cotain the word *'York'*)**

In [14]:
York_borough = ['North York', 'East York', 'York']
Yorkborough_data = Toronto_neigh[Toronto_neigh.Borough.isin(York_borough)].reset_index(drop=True)
print(Yorkborough_data.shape)
Yorkborough_data.tail(10)

(34, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
24,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
25,M6B,North York,Glencairn,43.709577,-79.445073
26,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
27,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
28,M6L,North York,"Maple Leaf Park, North Park, Upwood Park",43.713756,-79.490074
29,M6M,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",43.691116,-79.476013
30,M6N,York,"The Junction North, Runnymede",43.673185,-79.487262
31,M9L,North York,Humber Summit,43.756303,-79.565963
32,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
33,M9N,York,Weston,43.706876,-79.518188


**Determine the coordinates of York borough in Toronto**

In [15]:
address = 'York, Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of York, Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of York, Toronto are 43.6896191, -79.479188.


**Create a folium map of York boroughs in Toronto**

In [16]:
# create map of York boroughs using latitude and longitude values
map_Yorkborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Yorkborough_data['Latitude'], Yorkborough_data['Longitude'], Yorkborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Yorkborough)  
    
map_Yorkborough

In [17]:
#save map as html
map_Yorkborough.save(outfile= "Yorkborough_map.html")

**Define *Foursquare* credentials and version**

In [18]:
CLIENT_ID = 'F13HMB5L3JKK3YI4I2ID4SU2YC4FWSV4NNT1O5RLM4VDHKGJ' # your Foursquare ID
CLIENT_SECRET = 'BUL0XZLUH2TJLF2VIIDYV4LWNFXJ14YW5P32JSCP4RGLA1LA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: F13HMB5L3JKK3YI4I2ID4SU2YC4FWSV4NNT1O5RLM4VDHKGJ
CLIENT_SECRET:BUL0XZLUH2TJLF2VIIDYV4LWNFXJ14YW5P32JSCP4RGLA1LA


**Explore the first neighbourhood in our new dataframe**

In [19]:
Yorkborough_data.loc[0, 'Neighborhood']

'Hillcrest Village'

**Find the geographical coordinates of this neighbourhood**

In [20]:
neighborhood_latitude = Yorkborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Yorkborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Yorkborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hillcrest Village are 43.8037622, -79.3634517.


**Get a maximum of 100 top venues in Hillcrest Village within a radius of 600 metres**

In [21]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 600 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=F13HMB5L3JKK3YI4I2ID4SU2YC4FWSV4NNT1O5RLM4VDHKGJ&client_secret=BUL0XZLUH2TJLF2VIIDYV4LWNFXJ14YW5P32JSCP4RGLA1LA&v=20180605&ll=43.8037622,-79.3634517&radius=600&limit=100'

**Create *GET* request and display results**

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c4b3a936a60717af2e59677'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.80916220540001,
    'lng': -79.35598348245396},
   'sw': {'lat': 43.798362194599996, 'lng': -79.37091991754603}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad9dce6f964a520651b21e3',
       'name': "Eagle's Nest Golf Club",
       'location': {'address': '10000 Dufferin Rd',
        'lat': 43.805454826002794,
        'lng': -79.36418592243415,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.805454826002794,
          'lng': -79.36418592243415}],
        'distance': 197,
        'cc': 'CA',
        'city': 'T

**Extract venue categories from *Foursquare***

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

**Clean json and create a new *pandas* dataframe for nearby venues**

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Eagle's Nest Golf Club,Golf Course,43.805455,-79.364186
1,AY Jackson Pool,Pool,43.804515,-79.366138
2,Villa Madina,Mediterranean Restaurant,43.801685,-79.363938
3,Duncan Creek Park,Dog Run,43.805539,-79.360695
4,Hillcrest Tennis Club,Tennis Court,43.798561,-79.363506


**Print number of nearby venues**

In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


**Create a function to repeat the same process for all neighbourhoods in York boroughs**

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**Create a new dataframe for all venues in the neighbourhoods**

In [27]:
Yorkborough_venues = getNearbyVenues(names=Yorkborough_data['Neighborhood'],
                                   latitudes=Yorkborough_data['Latitude'],
                                   longitudes=Yorkborough_data['Longitude']
                                  )

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
Leaside
Thorncliffe Park
East Toronto
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Humewood-Cedarvale
Caledonia-Fairbanks
Maple Leaf Park, North Park, Upwood Park
Del Ray, Keelesdale, Mount Dennis, Silverthorn
The Junction North, Runnymede
Humber Summit
Emery, Humberlea
Weston


**Display size and head of the new dataframe**

In [28]:
print(Yorkborough_venues.shape)
Yorkborough_venues.head()

(328, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store


**Show how many venues were returned for each neighbourhood**

In [29]:
Yorkborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
"CFB Toronto, Downsview East",3,3,3,3,3,3
Caledonia-Fairbanks,6,6,6,6,6,6
"Del Ray, Keelesdale, Mount Dennis, Silverthorn",4,4,4,4,4,4
Don Mills North,7,7,7,7,7,7
Downsview Central,3,3,3,3,3,3
Downsview Northwest,5,5,5,5,5,5


**Display the total number of unique venue categories**

In [30]:
print('There are {} uniques categories.'.format(len(Yorkborough_venues['Venue Category'].unique())))

There are 125 uniques categories.


**Create a new dataframe with venue categories using one hot encoding**

In [31]:
# one hot encoding
Yorkborough_onehot = pd.get_dummies(Yorkborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Yorkborough_onehot['Neighborhood'] = Yorkborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Yorkborough_onehot.columns[-1]] + list(Yorkborough_onehot.columns[:-1])
Yorkborough_onehot = Yorkborough_onehot[fixed_columns]

Yorkborough_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Field,Fish & Chips Shop,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Home Service,Hotel,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Liquor Store,Lounge,Luggage Store,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Rock Climbing Spot,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [32]:
#display size of the new dataframe
Yorkborough_onehot.shape

(328, 126)

**Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [33]:
Yorkborough_grouped = Yorkborough_onehot.groupby('Neighborhood').mean().reset_index()
Yorkborough_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Field,Fish & Chips Shop,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Home Service,Hotel,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Liquor Store,Lounge,Luggage Store,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Rock Climbing Spot,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.058824,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.058824,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.117647,0.000000,0.00,0.00,0.000000,0.000000,0.058824,0.000000,0.00000,0.000000,0.058824,0.000000,0.00,0.000000,0.0,0.000000,0.058824,0.0,0.00000,0.000000,0.000000,0.000000,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.058824,0.058824,0.058824,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.058824,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.041667,0.000000,0.000000,0.0000

In [34]:
#display shape of the new dataframe
Yorkborough_grouped.shape

(33, 126)

**Display each neighbourhood with the top 5 most common venues**

In [35]:
num_top_venues = 5

for hood in Yorkborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Yorkborough_grouped[Yorkborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
                 venue  freq
0          Coffee Shop  0.12
1        Deli / Bodega  0.06
2           Restaurant  0.06
3  Fried Chicken Joint  0.06
4     Sushi Restaurant  0.06


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1  Japanese Restaurant  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4   Mexican Restaurant  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.08
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.08
3         Grocery Store  0.04
4               Butcher  0.04


----CFB Toronto, Downsview East----
        venue  freq
0     Airport  0.33
1        Park  0.33
2    Bus Stop  0.33
3        Pool  0.00
4  Playground  0.00


----Caledonia-Fairbanks----
                  venue  freq
0                  Park  0.33
1  Fast Food Restaurant  0.17
2         Women's Store  0.17
3              Pharmacy  0.17
4       

**Sort venues in descending order**

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**Create the new dataframe and display the top 10 venues for each neighborhood**

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Yorkborough_grouped['Neighborhood']

for ind in np.arange(Yorkborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Yorkborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Pizza Place,Fast Food Restaurant,Pet Store,Pharmacy,Restaurant,Deli / Bodega,Sandwich Place,Shopping Mall,Bridal Shop
1,Bayview Village,Chinese Restaurant,Bank,Japanese Restaurant,Café,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
2,"Bedford Park, Lawrence Manor East",Fast Food Restaurant,Italian Restaurant,Coffee Shop,Grocery Store,Comfort Food Restaurant,Pub,Butcher,Café,Pizza Place,Juice Bar
3,"CFB Toronto, Downsview East",Airport,Park,Bus Stop,Yoga Studio,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega
4,Caledonia-Fairbanks,Park,Women's Store,Pharmacy,Market,Fast Food Restaurant,Yoga Studio,Deli / Bodega,Dim Sum Restaurant,Dessert Shop,Department Store


**Cluster the neighbourhoods into 5 clusters using *k-means***

In [38]:
# set number of clusters
kclusters = 5

Yorkborough_grouped_clustering = Yorkborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Yorkborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 1, 1, 2, 2, 2, 2, 2], dtype=int32)

**Create a new dataframe with clusters and the top 10 venues in each neighbourhood**

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Yorkborough_merged = Yorkborough_data

# merge the dataframes to add latitude/longitude for each neighborhood
Yorkborough_merged = Yorkborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Yorkborough_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,2.0,Dog Run,Mediterranean Restaurant,Pool,Golf Course,Furniture / Home Store,Frozen Yogurt Shop,General Entertainment,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,2.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Asian Restaurant,Bakery,Tea Room,Toy / Game Store,Restaurant,Food Court,Video Game Store
2,M2K,North York,Bayview Village,43.786947,-79.385975,2.0,Chinese Restaurant,Bank,Japanese Restaurant,Café,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,4.0,Cafeteria,Yoga Studio,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Create a folium map with clusters**

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Yorkborough_merged['Latitude'], Yorkborough_merged['Longitude'], Yorkborough_merged['Neighborhood'], Yorkborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[-1],
        fill=True,
        fill_color=rainbow[-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Display Cluster 1**

In [43]:
Yorkborough_merged.loc[Yorkborough_merged['Cluster Labels'] == 0, Yorkborough_merged.columns[[2] + list(range(5, Yorkborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,"Emery, Humberlea",0.0,Baseball Field,Yoga Studio,Empanada Restaurant,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store


**Display Cluster 2**

In [44]:
Yorkborough_merged.loc[Yorkborough_merged['Cluster Labels'] == 1, Yorkborough_merged.columns[[1] + list(range(5, Yorkborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,North York,1.0,Park,Bank,Yoga Studio,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega
8,North York,1.0,Park,Food & Drink Shop,Fast Food Restaurant,Yoga Studio,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice
13,North York,1.0,Airport,Park,Bus Stop,Yoga Studio,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega
27,York,1.0,Park,Women's Store,Pharmacy,Market,Fast Food Restaurant,Yoga Studio,Deli / Bodega,Dim Sum Restaurant,Dessert Shop,Department Store
33,York,1.0,Park,Convenience Store,Yoga Studio,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store


**Display Cluster 3**

In [45]:
Yorkborough_merged.loc[Yorkborough_merged['Cluster Labels'] == 2, Yorkborough_merged.columns[[1] + list(range(5, Yorkborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2.0,Dog Run,Mediterranean Restaurant,Pool,Golf Course,Furniture / Home Store,Frozen Yogurt Shop,General Entertainment,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
1,North York,2.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Asian Restaurant,Bakery,Tea Room,Toy / Game Store,Restaurant,Food Court,Video Game Store
2,North York,2.0,Chinese Restaurant,Bank,Japanese Restaurant,Café,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
5,North York,2.0,Ramen Restaurant,Sandwich Place,Fast Food Restaurant,Restaurant,Japanese Restaurant,Pizza Place,Coffee Shop,Café,Movie Theater,Bubble Tea Shop
7,North York,2.0,Grocery Store,Butcher,Pharmacy,Coffee Shop,Pizza Place,Frozen Yogurt Shop,Dim Sum Restaurant,General Entertainment,Gastropub,Comfort Food Restaurant
9,North York,2.0,Basketball Court,Gym / Fitness Center,Pool,Caribbean Restaurant,Japanese Restaurant,Café,Baseball Field,Yoga Studio,Discount Store,Diner
10,North York,2.0,Grocery Store,Sporting Goods Shop,Gym,Asian Restaurant,Coffee Shop,Beer Store,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Dim Sum Restaurant
11,North York,2.0,Coffee Shop,Pizza Place,Fast Food Restaurant,Pet Store,Pharmacy,Restaurant,Deli / Bodega,Sandwich Place,Shopping Mall,Bridal Shop
12,North York,2.0,Coffee Shop,Miscellaneous Shop,Bar,Massage Studio,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice
14,North York,2.0,Grocery Store,Moving Target,Bank,Shopping Mall,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice


**Display Cluster 4**

In [46]:
Yorkborough_merged.loc[Yorkborough_merged['Cluster Labels'] == 3, Yorkborough_merged.columns[[1] + list(range(5, Yorkborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,North York,3.0,Pizza Place,Pub,Japanese Restaurant,Deli / Bodega,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Yoga Studio
30,York,3.0,Grocery Store,Bus Line,Convenience Store,Pizza Place,Gastropub,Furniture / Home Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Gift Shop
31,North York,3.0,Empanada Restaurant,Pizza Place,Gift Shop,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice


**Display Cluster 5**

In [47]:
Yorkborough_merged.loc[Yorkborough_merged['Cluster Labels'] == 4, Yorkborough_merged.columns[[1] + list(range(5, Yorkborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,4.0,Cafeteria,Yoga Studio,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store


### Thank you for you attention!